# Problem Set 7
#### Brennan McFarland

In this assignment, I design a tensorflow neural network for the BASF dataset and demonstrate its performance and how it changes in response to different hyperparameters and network architecture.  I demonstrate how the BASF dataset is very amenable to training a neural network on and that very low loss can be achieved even on a testing set, indicating effective generalization.

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

### Loading the Data

Before building the neural network, the BASF data must be preprocessed in a form the network can interpret.  First we load our inputs and targets from the corresponding csv files into numpy arrays:

In [2]:
raw_inputs = np.genfromtxt('basf_inputs_rand_normalized.csv', delimiter=',')
print(raw_inputs.shape)
print(raw_inputs)

(1029, 12)
[[3.         4.         1.         ... 0.57142857 0.40765391 1.        ]
 [7.         4.         1.         ... 1.         0.66139767 0.5       ]
 [2.         1.         0.         ... 0.57142857 0.71214642 0.33333333]
 ...
 [7.         4.         1.         ... 0.71428571 0.36439268 1.        ]
 [6.         4.         1.         ... 0.57142857 0.42845258 0.33333333]
 [1.         3.         0.         ... 0.57142857 0.68053245 0.5       ]]


In [3]:
raw_targets = np.genfromtxt('basf_targets_rand_normalized.csv', delimiter=',')
raw_targets = np.expand_dims(raw_targets, axis=1) # because tensorflow wants each column in an array even if only 1 value
print(raw_targets.shape)
print(raw_targets)

(1029, 1)
[[0.33097762]
 [0.26266196]
 [0.10600707]
 ...
 [0.34393404]
 [0.02120141]
 [0.29799764]]


### One-Hot Encoding

The first four features are categorical and thus should be converted from their current encodings as integers, as this implies an ordering relationship between their values.  This can be fixed by one-hot encoding, which transforms the data into vectors such that each categories' vector representation is equidistant from every other.  Here the first four columns of feature data are converted to one-hot values, done prior to splitting into training, validation and testing sets for the sake of reducing redundancy and for efficiency. The first four columns have 8, 4, 2, and 5 different possible values respectively, so that will be the dimension of each one-hot vector.

In [4]:
sess = tf.Session()
num_col_values = (8, 4, 2, 5)
new_cols = []
for col in range(4):
    col_values = raw_inputs[:, col]
    print(col_values)
    col_values = tf.cast(col_values, tf.int32)
    col_one_hot = tf.one_hot(col_values, num_col_values[col])
    print(sess.run(col_one_hot))
    new_cols.append(col_one_hot)
new_cols.append(raw_inputs[:, 4:])

[3. 7. 2. ... 7. 6. 1.]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 1. 0. ... 0. 0. 0.]]
[4. 4. 1. ... 4. 4. 3.]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 1. 0. 0.]
 ...
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 1.]]
[1. 1. 0. ... 1. 1. 0.]
[[0. 1.]
 [0. 1.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [1. 0.]]
[1. 3. 0. ... 3. 3. 2.]
[[0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]]


The inputs are evaluated after one-hot encoding to make sure they are correct.

In [5]:
inputs = tf.concat(new_cols, axis=1)
targets = raw_targets
print(sess.run(inputs))

[[0.         0.         0.         ... 0.5714286  0.4076539  1.        ]
 [0.         0.         0.         ... 1.         0.6613977  0.5       ]
 [0.         0.         1.         ... 0.5714286  0.7121464  0.33333334]
 ...
 [0.         0.         0.         ... 0.71428573 0.36439267 1.        ]
 [0.         0.         0.         ... 0.5714286  0.42845258 0.33333334]
 [0.         1.         0.         ... 0.5714286  0.68053246 0.5       ]]


### Splitting the Dataset

We'll reserve 80% of the data for training and 10% for validation and testing, respectively, per convention.  The dataset is split into independent training, validation and test sets:

In [6]:
training_bound = int(int(inputs.shape[0])*.8)
validation_bound = int(int(inputs.shape[0])*.9)
training_inputs, training_targets = inputs[:training_bound], targets[:training_bound]
validation_inputs, validation_targets = inputs[training_bound:validation_bound], targets[training_bound:validation_bound]
testing_inputs, testing_targets = inputs[validation_bound:], targets[validation_bound:]
print(int(training_inputs.shape[0])+ int(validation_inputs.shape[0]) + int(testing_inputs.shape[0])) # verify number examples preserved
print(training_inputs)
print(validation_inputs)
print(testing_inputs)

1029
Tensor("strided_slice:0", shape=(823, 27), dtype=float32)
Tensor("strided_slice_1:0", shape=(103, 27), dtype=float32)
Tensor("strided_slice_2:0", shape=(103, 27), dtype=float32)


### Defining the Network

In [8]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


Now we can actually define our network.  First I will define a reasonable guess as to what a good network might be, train and evaluate it, and then make modifications to it until a more optimal network is achieved.

We can start with a network of two dense layers with 16 nodes each, followed by a single node layer.  This will be necessary for all BASF regression networks as the output is one-dimensional.  The activation functions for the multineuron layers are rectified linear unit, but we will try a linear activation function for the third dense layer, which is a reasonable choice since it is the last layer and this could help the network automatically scale the output appropriately:

In [9]:
model = Sequential()
model.add(Dense(16, input_dim=int(training_inputs[0].shape[0]), activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='linear'))

Other optimizers are available with the Keras API, but stochastic gradient descent is simple and efficient, so it seems a good choice for the network:

In [10]:
# use stochastic gradient descent as the optimizer
optimizer = SGD(lr=.1)

Rather than have the network train for a fixed number of iterations, we insert the early stopping callback into the network fitting function.  The number of training epochs is capped at 1000, but early stopping will halt the training process once the loss on the validation set stops improving.  This will both prevent overfitting and allow for more accurate comparison between different versions of the network, as each one will have trained to what is hopefully a close to optimal state, as opposed to an arbitrary stopping point:

In [11]:
# early stopping callback to stop when validation loss stops improving
early_stopping = EarlyStopping(monitor='val_loss')

Since this is a regression and not a classification problem, the network will perform better using mean squared error loss as opposed to, say, binary crossentropy, which is designed for evaluating the number of answers exactly correct:

In [12]:
model.compile(loss='mean_squared_error', optimizer=optimizer)

In [13]:
steps_per_epoch = 10
validation_steps = 10
testing_steps = 10

Finally, we can fit our initial model on the training data, running until the validation loss stops decreasing at the 38th epoch:

In [14]:
print(training_targets.shape)
model.fit(
    training_inputs,
    training_targets,
    validation_data = (validation_inputs, validation_targets),
    steps_per_epoch = steps_per_epoch,
    validation_steps = validation_steps,
    callbacks=[early_stopping],
    epochs=1000)

(823, 1)
Train on 823 samples, validate on 103 samples
Epoch 1/1000
10/10 [==============================] - 1s 51ms/step - loss: 0.0784 - val_loss: 0.0525
Epoch 2/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0425 - val_loss: 0.0431
Epoch 3/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0376 - val_loss: 0.0392
Epoch 4/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0348 - val_loss: 0.0368
Epoch 5/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0327 - val_loss: 0.0348
Epoch 6/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0308 - val_loss: 0.0327
Epoch 7/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0291 - val_loss: 0.0307
Epoch 8/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0274 - val_loss: 0.0289
Epoch 9/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0260 - val_loss: 0.0272
Epoch 10/1000
10/10 [=======================

To truly evaluate the generalization capability of the network, we have to do so on new data, and so we measure the loss of the network on the testing portion of our dataset, obtaining a loss around .012:

In [15]:
testing_loss = model.evaluate(
    testing_inputs,
    testing_targets,
    steps = testing_steps,
    verbose = 0)
print("Post-training loss on test data: ", testing_loss)

Post-training loss on test data:  0.011815818957984447


This is not terrible, but given that the target values are on a scale of 0 to 1, we can do much better.  Next we try the same approach with one modification: changing the activation function of the last layer from linear to a ReLU.  Although the model definition and fitting process could go in a function, I chose to repeat it each time for the sake of illustration.  This time, the network runs significantly longer, for 318 epochs, and obtains a loss on the testing data of .004, about a third the loss of before.  This is a substantial improvement gained by changing only one small aspect of the network, albeit while incurring significant training time (though the network takes only a few seconds to train anyway, so this isn't a big deal):

In [16]:
modelB = Sequential()
modelB.add(Dense(16, input_dim=int(training_inputs[0].shape[0]), activation='relu'))
modelB.add(Dense(16, activation='relu'))
modelB.add(Dense(1, activation='relu'))
modelB.compile(loss='mean_squared_error', optimizer=optimizer)
modelB.fit(
    training_inputs,
    training_targets,
    validation_data = (validation_inputs, validation_targets),
    steps_per_epoch = steps_per_epoch,
    validation_steps = validation_steps,
    callbacks=[early_stopping],
    epochs=1000)
testing_loss = modelB.evaluate(
    testing_inputs,
    testing_targets,
    steps = testing_steps,
    verbose = 0)
print("Post-training loss on test data: ", testing_loss)

Train on 823 samples, validate on 103 samples
Epoch 1/1000
10/10 [==============================] - 0s 26ms/step - loss: 0.0484 - val_loss: 0.0561
Epoch 2/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0430 - val_loss: 0.0526
Epoch 3/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0406 - val_loss: 0.0502
Epoch 4/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0389 - val_loss: 0.0484
Epoch 5/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0376 - val_loss: 0.0469
Epoch 6/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0366 - val_loss: 0.0457
Epoch 7/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0358 - val_loss: 0.0446
Epoch 8/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0351 - val_loss: 0.0435
Epoch 9/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0343 - val_loss: 0.0424
Epoch 10/1000
10/10 [==============================] 

10/10 [==============================] - 0s 4ms/step - loss: 0.0067 - val_loss: 0.0082
Epoch 82/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0066 - val_loss: 0.0080
Epoch 83/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0065 - val_loss: 0.0079
Epoch 84/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0065 - val_loss: 0.0078
Epoch 85/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0064 - val_loss: 0.0077
Epoch 86/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0063 - val_loss: 0.0076
Epoch 87/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0062 - val_loss: 0.0075
Epoch 88/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0062 - val_loss: 0.0074
Epoch 89/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0061 - val_loss: 0.0073
Epoch 90/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0060 - val_loss: 0.0072
Epoc

10/10 [==============================] - 0s 3ms/step - loss: 0.0043 - val_loss: 0.0048
Epoch 162/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0043 - val_loss: 0.0047
Epoch 163/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0043 - val_loss: 0.0047
Epoch 164/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0043 - val_loss: 0.0047
Epoch 165/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0043 - val_loss: 0.0047
Epoch 166/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0043 - val_loss: 0.0047
Epoch 167/1000
10/10 [==============================] - ETA: 0s - loss: 0.004 - 0s 3ms/step - loss: 0.0043 - val_loss: 0.0047
Epoch 168/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0043 - val_loss: 0.0047
Epoch 169/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0042 - val_loss: 0.0047
Epoch 170/1000
10/10 [==============================] - 0s 3ms/step - los

Epoch 241/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0039 - val_loss: 0.0043
Epoch 242/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0039 - val_loss: 0.0043
Epoch 243/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0039 - val_loss: 0.0043
Epoch 244/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0039 - val_loss: 0.0043
Epoch 245/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0039 - val_loss: 0.0043
Epoch 246/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0039 - val_loss: 0.0043
Epoch 247/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0039 - val_loss: 0.0043
Epoch 248/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0039 - val_loss: 0.0043
Epoch 249/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0039 - val_loss: 0.0043
Epoch 250/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0039

We can do even better with some further modifications.  Next we try removing a layer entirely, leaving only the first multinode layer and the single-node layer.  This time, the network runs for only 244 iterations, but while the validation set loss appears the same as with model B, the testing set loss is greater, at a little over .004.  Note that running the network multiple times yeilds slightly different (though still consistent) results, so running models B or C again may reverse their relative performance, but it is safe to say that this time we gain no significant benefit from ablating a hidden layer and in fact it may have hurt performance:

In [20]:
modelC = Sequential()
modelC.add(Dense(16, input_dim=int(training_inputs[0].shape[0]), activation='relu'))
modelC.add(Dense(1, activation='relu'))
modelC.compile(loss='mean_squared_error', optimizer=optimizer)
modelC.fit(
    training_inputs,
    training_targets,
    validation_data = (validation_inputs, validation_targets),
    steps_per_epoch = steps_per_epoch,
    validation_steps = validation_steps,
    callbacks=[early_stopping],
    epochs=1000)
testing_loss = modelC.evaluate(
    testing_inputs,
    testing_targets,
    steps = testing_steps,
    verbose = 0)
print("Post-training loss on test data: ", testing_loss)

Train on 823 samples, validate on 103 samples
Epoch 1/1000
10/10 [==============================] - 0s 31ms/step - loss: 0.0933 - val_loss: 0.0740
Epoch 2/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0540 - val_loss: 0.0639
Epoch 3/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0464 - val_loss: 0.0550
Epoch 4/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0412 - val_loss: 0.0480
Epoch 5/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0371 - val_loss: 0.0428
Epoch 6/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0338 - val_loss: 0.0388
Epoch 7/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0311 - val_loss: 0.0355
Epoch 8/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0290 - val_loss: 0.0328
Epoch 9/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0272 - val_loss: 0.0305
Epoch 10/1000
10/10 [==============================] 

10/10 [==============================] - 0s 3ms/step - loss: 0.0058 - val_loss: 0.0059
Epoch 82/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0057 - val_loss: 0.0059
Epoch 83/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0057 - val_loss: 0.0058
Epoch 84/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0056 - val_loss: 0.0058
Epoch 85/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0056 - val_loss: 0.0057
Epoch 86/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0055 - val_loss: 0.0057
Epoch 87/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0055 - val_loss: 0.0057
Epoch 88/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0055 - val_loss: 0.0056
Epoch 89/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0054 - val_loss: 0.0056
Epoch 90/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0054 - val_loss: 0.0055
Epoc

10/10 [==============================] - 0s 3ms/step - loss: 0.0043 - val_loss: 0.0044
Epoch 162/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0043 - val_loss: 0.0044
Epoch 163/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0043 - val_loss: 0.0044
Epoch 164/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0043 - val_loss: 0.0044
Epoch 165/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0043 - val_loss: 0.0044
Epoch 166/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0043 - val_loss: 0.0044
Epoch 167/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0043 - val_loss: 0.0044
Epoch 168/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0043 - val_loss: 0.0044
Epoch 169/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0043 - val_loss: 0.0044
Epoch 170/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0043 - val_loss: 0.

10/10 [==============================] - 0s 3ms/step - loss: 0.0040 - val_loss: 0.0042
Epoch 242/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0040 - val_loss: 0.0042
Epoch 243/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0040 - val_loss: 0.0042
Epoch 244/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0040 - val_loss: 0.0042
Post-training loss on test data:  0.004592417739331722


Instead of removing layers entirely, then, we can try changing the number of nodes in each layer (barring the last one, as stated above).  The next model is the same as model B, except the first two layers now have 8 instead of 16 neurons each.  This actually results in a very significant improvement, with a loss of .002, about half that of model B, on the test set, again at the cost of more epochs, a total of 849 in this case.  The most likely explanation for this is just that the network is capable of more accurately approximating the targets given an indefinite amount of time to train and so the early stopping does not take effect until much later, and indeed we see that at the number of epochs when B stopped training the training and validation loss are about the same:

In [18]:
modelD = Sequential()
modelD.add(Dense(8, input_dim=int(training_inputs[0].shape[0]), activation='relu'))
modelD.add(Dense(8, activation='relu'))
modelD.add(Dense(1, activation='relu'))
modelD.compile(loss='mean_squared_error', optimizer=optimizer)
modelD.fit(
    training_inputs,
    training_targets,
    validation_data = (validation_inputs, validation_targets),
    steps_per_epoch = steps_per_epoch,
    validation_steps = validation_steps,
    callbacks=[early_stopping],
    epochs=1000)
testing_loss = modelD.evaluate(
    testing_inputs,
    testing_targets,
    steps = testing_steps,
    verbose = 0)
print("Post-training loss on test data: ", testing_loss)

Train on 823 samples, validate on 103 samples
Epoch 1/1000
10/10 [==============================] - 0s 29ms/step - loss: 0.0872 - val_loss: 0.0888
Epoch 2/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0680 - val_loss: 0.0811
Epoch 3/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0633 - val_loss: 0.0769
Epoch 4/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0598 - val_loss: 0.0722
Epoch 5/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0563 - val_loss: 0.0663
Epoch 6/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0520 - val_loss: 0.0611
Epoch 7/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0483 - val_loss: 0.0575
Epoch 8/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0455 - val_loss: 0.0555
Epoch 9/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0440 - val_loss: 0.0540
Epoch 10/1000
10/10 [==============================] 

10/10 [==============================] - 0s 3ms/step - loss: 0.0270 - val_loss: 0.0306
Epoch 82/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0268 - val_loss: 0.0304
Epoch 83/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0266 - val_loss: 0.0302
Epoch 84/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0264 - val_loss: 0.0300
Epoch 85/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0263 - val_loss: 0.0298
Epoch 86/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0261 - val_loss: 0.0296
Epoch 87/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0259 - val_loss: 0.0294
Epoch 88/1000
10/10 [==============================] - ETA: 0s - loss: 0.025 - 0s 3ms/step - loss: 0.0257 - val_loss: 0.0291
Epoch 89/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0255 - val_loss: 0.0289
Epoch 90/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0253

10/10 [==============================] - 0s 3ms/step - loss: 0.0097 - val_loss: 0.0108
Epoch 162/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0095 - val_loss: 0.0106
Epoch 163/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0093 - val_loss: 0.0104
Epoch 164/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0091 - val_loss: 0.0102
Epoch 165/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0089 - val_loss: 0.0099
Epoch 166/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0088 - val_loss: 0.0097
Epoch 167/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0086 - val_loss: 0.0095
Epoch 168/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0084 - val_loss: 0.0092
Epoch 169/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0082 - val_loss: 0.0090
Epoch 170/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0080 - val_loss: 0.

10/10 [==============================] - 0s 3ms/step - loss: 0.0041 - val_loss: 0.0047
Epoch 242/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0041 - val_loss: 0.0047
Epoch 243/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0041 - val_loss: 0.0047
Epoch 244/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0041 - val_loss: 0.0047
Epoch 245/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0041 - val_loss: 0.0047
Epoch 246/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0041 - val_loss: 0.0047
Epoch 247/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0041 - val_loss: 0.0047
Epoch 248/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0041 - val_loss: 0.0047
Epoch 249/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0041 - val_loss: 0.0047
Epoch 250/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0041 - val_loss: 0.

Epoch 321/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0036 - val_loss: 0.0041
Epoch 322/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0036 - val_loss: 0.0041
Epoch 323/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0036 - val_loss: 0.0041
Epoch 324/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0036 - val_loss: 0.0041
Epoch 325/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0036 - val_loss: 0.0041
Epoch 326/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0036 - val_loss: 0.0041
Epoch 327/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0036 - val_loss: 0.0041
Epoch 328/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0036 - val_loss: 0.0041
Epoch 329/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0036 - val_loss: 0.0041
Epoch 330/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0036

...

10/10 [==============================] - 0s 4ms/step - loss: 0.0032 - val_loss: 0.0036
Epoch 482/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0032 - val_loss: 0.0036
Epoch 483/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0032 - val_loss: 0.0036
Epoch 484/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0032 - val_loss: 0.0036
Epoch 485/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0032 - val_loss: 0.0036
Epoch 486/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0032 - val_loss: 0.0036
Epoch 487/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0032 - val_loss: 0.0036
Epoch 488/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0032 - val_loss: 0.0036
Epoch 489/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0032 - val_loss: 0.0036
Epoch 490/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0032 - val_loss: 0.

10/10 [==============================] - 0s 3ms/step - loss: 0.0030 - val_loss: 0.0034
Epoch 562/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0030 - val_loss: 0.0034
Epoch 563/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0030 - val_loss: 0.0034
Epoch 564/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0030 - val_loss: 0.0034
Epoch 565/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0030 - val_loss: 0.0034
Epoch 566/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0030 - val_loss: 0.0034
Epoch 567/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0030 - val_loss: 0.0034
Epoch 568/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0030 - val_loss: 0.0034
Epoch 569/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0030 - val_loss: 0.0034
Epoch 570/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0030 - val_loss: 0.

10/10 [==============================] - 0s 3ms/step - loss: 0.0030 - val_loss: 0.0033
Epoch 642/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0030 - val_loss: 0.0033
Epoch 643/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0030 - val_loss: 0.0033
Epoch 644/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0030 - val_loss: 0.0033
Epoch 645/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0030 - val_loss: 0.0033
Epoch 646/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0030 - val_loss: 0.0033
Epoch 647/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0030 - val_loss: 0.0033
Epoch 648/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0030 - val_loss: 0.0033
Epoch 649/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0030 - val_loss: 0.0033
Epoch 650/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0030 - val_loss: 0.

10/10 [==============================] - 0s 4ms/step - loss: 0.0028 - val_loss: 0.0029
Epoch 722/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0028 - val_loss: 0.0029
Epoch 723/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0028 - val_loss: 0.0029
Epoch 724/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0028 - val_loss: 0.0029
Epoch 725/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0028 - val_loss: 0.0029
Epoch 726/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0028 - val_loss: 0.0029
Epoch 727/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0028 - val_loss: 0.0029
Epoch 728/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0028 - val_loss: 0.0029
Epoch 729/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0028 - val_loss: 0.0029
Epoch 730/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0028 - val_loss: 0.

10/10 [==============================] - 0s 4ms/step - loss: 0.0027 - val_loss: 0.0027
Epoch 802/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0027 - val_loss: 0.0027
Epoch 803/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0027 - val_loss: 0.0027
Epoch 804/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0027 - val_loss: 0.0027
Epoch 805/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0027 - val_loss: 0.0027
Epoch 806/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0027 - val_loss: 0.0027
Epoch 807/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0027 - val_loss: 0.0027
Epoch 808/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0027 - val_loss: 0.0027
Epoch 809/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0027 - val_loss: 0.0027
Epoch 810/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0027 - val_loss: 0.

Finally, we demonstrate that there is in fact a limit to how many nodes we can remove.  Cutting the number of neurons per multineuron layer by only two more, as in model E, results in a significantly greater testing loss of around .004, similar again to model B, after the early stopping kicks in at 166 iterations:

In [19]:
modelE = Sequential()
modelE.add(Dense(6, input_dim=int(training_inputs[0].shape[0]), activation='relu'))
modelE.add(Dense(6, activation='relu'))
modelE.add(Dense(1, activation='relu'))
modelE.compile(loss='mean_squared_error', optimizer=optimizer)
modelE.fit(
    training_inputs,
    training_targets,
    validation_data = (validation_inputs, validation_targets),
    steps_per_epoch = steps_per_epoch,
    validation_steps = validation_steps,
    callbacks=[early_stopping],
    epochs=1000)
testing_loss = modelE.evaluate(
    testing_inputs,
    testing_targets,
    steps = testing_steps,
    verbose = 0)
print("Post-training loss on test data: ", testing_loss)

Train on 823 samples, validate on 103 samples
Epoch 1/1000
10/10 [==============================] - 0s 32ms/step - loss: 0.0590 - val_loss: 0.0556
Epoch 2/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0428 - val_loss: 0.0548
Epoch 3/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0424 - val_loss: 0.0543
Epoch 4/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0421 - val_loss: 0.0540
Epoch 5/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0419 - val_loss: 0.0537
Epoch 6/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0417 - val_loss: 0.0534
Epoch 7/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0415 - val_loss: 0.0531
Epoch 8/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0413 - val_loss: 0.0527
Epoch 9/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0411 - val_loss: 0.0524
Epoch 10/1000
10/10 [==============================] 

10/10 [==============================] - 0s 3ms/step - loss: 0.0097 - val_loss: 0.0110
Epoch 82/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0095 - val_loss: 0.0107
Epoch 83/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0093 - val_loss: 0.0105
Epoch 84/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0091 - val_loss: 0.0102
Epoch 85/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0089 - val_loss: 0.0100
Epoch 86/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0087 - val_loss: 0.0098
Epoch 87/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.0085 - val_loss: 0.0096
Epoch 88/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0084 - val_loss: 0.0094
Epoch 89/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0082 - val_loss: 0.0092
Epoch 90/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0081 - val_loss: 0.0090
Epoc

10/10 [==============================] - 0s 3ms/step - loss: 0.0045 - val_loss: 0.0048
Epoch 162/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0045 - val_loss: 0.0047
Epoch 163/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0045 - val_loss: 0.0047
Epoch 164/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0045 - val_loss: 0.0047
Epoch 165/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0045 - val_loss: 0.0047
Epoch 166/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0045 - val_loss: 0.0047
Post-training loss on test data:  0.003980869427323341


Thus, further experimentation always has the possibility to result in a better model, but it is safe to say that model D, with a loss of around .002, is a good network architecture for this particular dataset with fairly well-tuned hyperparameters, and furthermore that the BASF dataset itself is well-suited to training a generalizable neural network.